In [1]:
import numpy as np
import pandas as pd
import zipfile
import gc
gc.enable()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = "order_products__train"
products = 'products'
departments = 'departments'
aisles = 'aisles'
prior= "order_products__prior"
orders='orders'
path="/content/drive/MyDrive/Data_btp2/"
orders = zipfile.ZipFile(path+orders+".zip","r")
products = zipfile.ZipFile(path+products+".zip","r")
departments = zipfile.ZipFile(path+departments+".zip","r")
aisles = zipfile.ZipFile(path+aisles+".zip","r")
train = zipfile.ZipFile(path+train+".zip","r")
prior=zipfile.ZipFile(path+prior+".zip","r")

In [4]:
train = pd.read_csv(train.open('order_products__train.csv'))
prior =pd.read_csv(prior.open('order_products__prior.csv'))
products = pd.read_csv(products.open('products.csv'), usecols=['product_id', 'aisle_id', 'department_id'])
departments = pd.read_csv(departments.open('departments.csv'))
aisles = pd.read_csv(aisles.open('aisles.csv'))
orders = pd.read_csv(orders.open('orders.csv'))

In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


In [6]:
orders.eval_set = orders.eval_set.replace({'prior': 0, 'train': 1, 'test':2})
orders.days_since_prior_order = orders.days_since_prior_order.fillna(30)

In [7]:
prior_orders = pd.merge(orders,prior, on='order_id', how='inner')
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,0,1,2,8,30.0,196,1,0
1,2539329,1,0,1,2,8,30.0,14084,2,0
2,2539329,1,0,1,2,8,30.0,12427,3,0
3,2539329,1,0,1,2,8,30.0,26088,4,0
4,2539329,1,0,1,2,8,30.0,26405,5,0


In [8]:
del prior
gc.collect()


23

In [9]:
users = prior_orders.groupby(by='user_id')['order_number'].aggregate('max').to_frame('u_num_of_orders').reset_index()
users.head()

,user_id,u_num_of_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [10]:
total_prd_per_order = prior_orders.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').to_frame('total_products_per_order').reset_index()

In [11]:
avg_products = total_prd_per_order.groupby(by=['user_id'])['total_products_per_order'].mean().to_frame('u_avg_prd').reset_index()
avg_products.head()

,user_id,u_avg_prd
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000


In [12]:
del [total_prd_per_order]
gc.collect()

avg_products.head()

,user_id,u_avg_prd
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000


In [13]:
avg_products.head()

,user_id,u_avg_prd
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000


In [14]:
from scipy import stats

dow = prior_orders.groupby(by=['user_id'])['order_dow'].aggregate(lambda x : stats.mode(x)[0]).to_frame('dow_u_most_orders')

dow = dow.reset_index()
dow.head()

,user_id,dow_u_most_orders
0,1,4
1,2,2
2,3,0
3,4,4
4,5,3


In [15]:
hod = prior_orders.groupby(by=['user_id'])['order_hour_of_day'].aggregate(lambda x : stats.mode(x)[0]).to_frame('hod_u_most_orders')
#resetting the index
hod = hod.reset_index()
hod.head()

,user_id,hod_u_most_orders
0,1,7
1,2,9
2,3,16
3,4,15
4,5,18


In [16]:
reorder_u = prior_orders.groupby(by='user_id')['reordered'].aggregate('mean').to_frame('u_reorder_ratio').reset_index()
#changing the dtype.
reorder_u['u_reorder_ratio'] = reorder_u['u_reorder_ratio'].astype(np.float16)
reorder_u.head()

,user_id,u_reorder_ratio
0,1,0.694824
1,2,0.476807
2,3,0.625000
3,4,0.055542
4,5,0.378418


In [17]:
prior_orders.days_since_prior_order.fillna(0, inplace=True)

In [18]:
avg_days = prior_orders.groupby(by='user_id')['days_since_prior_order'].aggregate('mean').to_frame('average_days_between_orders')
#resetting index
avg_days = avg_days.reset_index()
avg_days.head()

,user_id,average_days_between_orders
0,1,21.084746
1,2,16.902564
2,3,13.590909
3,4,18.611111
4,5,19.108108


In [19]:
total_item = prior_orders.groupby(by='user_id').size().to_frame('u_total_items_bought').astype(np.int16)
total_item.head()

,u_total_items_bought
user_id,
1,59
2,195
3,88
4,18
5,37


In [20]:
users = users.merge(avg_products, on='user_id', how='left')
#merging users df with dow
users = users.merge(dow, on='user_id', how='left')
#merging users df with hod
users = users.merge(hod, on='user_id', how='left')
#merging users df with reorder_u
users = users.merge(reorder_u, on='user_id', how='left')
#merging users df with avg_days
users = users.merge(avg_days, on='user_id', how='left')
#merging total_item df with reorder_u
users = users.merge(total_item, on='user_id', how='left')

users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_u_most_orders,hod_u_most_orders,u_reorder_ratio,average_days_between_orders,u_total_items_bought
0,1,10,5.900000,4,7,0.694824,21.084746,59
1,2,14,13.928571,2,9,0.476807,16.902564,195
2,3,12,7.333333,0,16,0.625000,13.590909,88
3,4,5,3.600000,4,15,0.055542,18.611111,18
4,5,4,9.250000,3,18,0.378418,19.108108,37


In [21]:
del [reorder_u, dow, hod, avg_products, avg_days, total_item]
gc.collect()

24

In [22]:
prd = prior_orders.groupby(by='product_id')['order_id'].aggregate('count').to_frame('p_num_of_times').reset_index()
# prd['p_num_of_times'] = prd['p_num_of_times'].astype(np.uint16)
prd.head()

,product_id,p_num_of_times
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [23]:
reorder_p = prior_orders.groupby(by='product_id')['reordered'].aggregate('mean').to_frame('p_reorder_ratio').reset_index()
# #changing dtype
# reorder_p['p_reorder_ratio'] = reorder_p['p_reorder_ratio'].astype(np.float16)
reorder_p.head()

,product_id,p_reorder_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000


In [24]:
add_to_cart = prior_orders.groupby(by='product_id')['add_to_cart_order'].aggregate('mean').to_frame('p_avg_cart_position').reset_index()
# #changing the dtype
# add_to_cart['p_avg_cart_position'] = add_to_cart['p_avg_cart_position'].astype(np.float16)
add_to_cart.head()

,product_id,p_avg_cart_position
0,1,5.801836
1,2,9.888889
2,3,6.415162
3,4,9.507599
4,5,6.466667


In [25]:
prd = prd.merge(reorder_p, on='product_id', how='left')

#merging add_to_cart with prd.
prd = prd.merge(add_to_cart, on='product_id', how='left')

#deleting unwanted df.
del [reorder_p, add_to_cart]
gc.collect()

36

In [26]:
prd.head()

,product_id,p_num_of_times,p_reorder_ratio,p_avg_cart_position
0,1,1852,0.613391,5.801836
1,2,90,0.133333,9.888889
2,3,277,0.732852,6.415162
3,4,329,0.446809,9.507599
4,5,15,0.600000,6.466667


In [27]:
uxp = prior_orders.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').to_frame('uxp_times_bought')
uxp=uxp.reset_index()
uxp.head()

,user_id,product_id,uxp_times_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [28]:
times=prior_orders.groupby(by=['user_id','product_id'])['order_id'].aggregate('count').to_frame('times_bought')
times=times.reset_index()
times.head()

,user_id,product_id,times_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [29]:
total_orders=prior_orders.groupby('user_id')['order_number'].max().to_frame('total_orders').reset_index()
total_orders.head()

,user_id,total_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [30]:
first_order_num=prior_orders.groupby(by=['user_id','product_id'])['order_number'].aggregate('min').to_frame('first_order_num')
first_order_num=first_order_num.reset_index()
first_order_num.head()

,user_id,product_id,first_order_num
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [31]:
span = pd.merge(total_orders, first_order_num, on='user_id', how='right')
span.head()

,user_id,total_orders,product_id,first_order_num
0,1,10,196,1
1,1,10,10258,2
2,1,10,10326,5
3,1,10,12427,1
4,1,10,13032,2


In [32]:
span['Order_Range_D'] = span.total_orders - span.first_order_num + 1
span.head()

,user_id,total_orders,product_id,first_order_num,Order_Range_D
0,1,10,196,1,10
1,1,10,10258,2,9
2,1,10,10326,5,6
3,1,10,12427,1,10
4,1,10,13032,2,9


In [33]:
uxp_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
uxp_ratio.head()

,user_id,product_id,times_bought,total_orders,first_order_num,Order_Range_D
0,1,196,10,10,1,10
1,1,10258,9,10,2,9
2,1,10326,1,10,5,6
3,1,12427,10,10,1,10
4,1,13032,3,10,2,9


In [34]:
uxp_ratio['uxp_reorder_ratio'] = uxp_ratio.times_bought / uxp_ratio.Order_Range_D
uxp_ratio.head()

,user_id,product_id,times_bought,total_orders,first_order_num,Order_Range_D,uxp_reorder_ratio
0,1,196,10,10,1,10,1.000000
1,1,10258,9,10,2,9,1.000000
2,1,10326,1,10,5,6,0.166667
3,1,12427,10,10,1,10,1.000000
4,1,13032,3,10,2,9,0.333333


In [35]:
uxp_ratio.drop(['times_bought', 'total_orders', 'first_order_num', 'Order_Range_D'], axis=1, inplace=True)
uxp_ratio.head()

,user_id,product_id,uxp_reorder_ratio
0,1,196,1.000000
1,1,10258,1.000000
2,1,10326,0.166667
3,1,12427,1.000000
4,1,13032,0.333333


In [36]:
del [times, span, first_order_num, total_orders]
gc.collect()

92

In [37]:
uxp = uxp.merge(uxp_ratio, on=['user_id', 'product_id'], how='left')
del uxp_ratio
gc.collect()

0

In [38]:
uxp.head()

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio
0,1,196,10,1.000000
1,1,10258,9,1.000000
2,1,10326,1,0.166667
3,1,12427,10,1.000000
4,1,13032,3,0.333333


In [39]:
prior_orders['order_number_back'] = prior_orders.groupby(by=['user_id'])['order_number'].transform(max) - prior_orders.order_number + 1
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
0,2539329,1,0,1,2,8,30.0,196,1,0,10
1,2539329,1,0,1,2,8,30.0,14084,2,0,10
2,2539329,1,0,1,2,8,30.0,12427,3,0,10
3,2539329,1,0,1,2,8,30.0,26088,4,0,10
4,2539329,1,0,1,2,8,30.0,26405,5,0,10


In [40]:
temp = prior_orders.loc[prior_orders.order_number_back <= 5]
temp.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
29,3367565,1,0,6,2,7,19.0,196,1,1,5
30,3367565,1,0,6,2,7,19.0,12427,2,1,5
31,3367565,1,0,6,2,7,19.0,10258,3,1,5
32,3367565,1,0,6,2,7,19.0,25133,4,1,5
33,550135,1,0,7,1,9,20.0,196,1,1,4


In [41]:
last_five = temp.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').to_frame('uxp_last_five').reset_index()
last_five.head()

,user_id,product_id,uxp_last_five
0,1,196,5
1,1,10258,5
2,1,12427,5
3,1,13032,2
4,1,25133,5


In [42]:
last_five['uxp_ratio_last_five'] = last_five.uxp_last_five / 5.0
last_five.head()

,user_id,product_id,uxp_last_five,uxp_ratio_last_five
0,1,196,5,1.0
1,1,10258,5,1.0
2,1,12427,5,1.0
3,1,13032,2,0.4
4,1,25133,5,1.0


In [43]:
uxp = uxp.merge(last_five, on=['user_id', 'product_id'], how='left')

del [last_five, temp]
gc.collect()
uxp.head()

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio,uxp_last_five,uxp_ratio_last_five
0,1,196,10,1.000000,5.0,1.0
1,1,10258,9,1.000000,5.0,1.0
2,1,10326,1,0.166667,NaN,NaN
3,1,12427,10,1.000000,5.0,1.0
4,1,13032,3,0.333333,2.0,0.4


In [44]:
uxp.fillna(0, inplace=True)
uxp.head(10)

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio,uxp_last_five,uxp_ratio_last_five
0,1,196,10,1.000000,5.0,1.0
1,1,10258,9,1.000000,5.0,1.0
2,1,10326,1,0.166667,0.0,0.0
3,1,12427,10,1.000000,5.0,1.0
4,1,13032,3,0.333333,2.0,0.4
5,1,13176,2,0.222222,0.0,0.0
6,1,14084,1,0.100000,0.0,0.0
7,1,17122,1,0.166667,0.0,0.0
8,1,25133,8,1.000000,5.0,1.0
9,1,26088,2,0.200000,0.0,0.0


In [45]:
uxp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   user_id              int64  
 1   product_id           int64  
 2   uxp_times_bought     int64  
 3   uxp_reorder_ratio    float64
 4   uxp_last_five        float64
 5   uxp_ratio_last_five  float64
dtypes: float64(3), int64(3)
memory usage: 710.7 MB


In [ ]:
data = uxp.merge(users, on='user_id', how='left')
data.head()

In [ ]:
data = data.merge(prd, on='product_id', how='left')
data.head()

In [ ]:
del [users, prd, uxp]
gc.collect()

In [ ]:
data.shape

In [ ]:
orders_future = orders.loc[((orders.eval_set == 1) | (orders.eval_set == 2)), ['user_id', 'eval_set', 'order_id']]
orders_future.head()

In [ ]:
data = data.merge(orders_future, on='user_id', how='left')
data.tail()

In [ ]:
data_train = data[data.eval_set == 1]
data_train.head()

In [ ]:
data_train = data_train.merge(train[['product_id', 'order_id', 'reordered']], on=['product_id', 'order_id'], how='left')
data_train.head()

In [ ]:
data_train.reordered.fillna(0, inplace=True)

In [ ]:
data_train.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [ ]:
data_train.head()

In [ ]:
data_test = data[data.eval_set == 2]
data_test.head()

In [ ]:
data_test.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [ ]:
data_train.shape, data_test.shape